In [1]:
import numpy as np
from scipy.optimize import linprog
import plotly.graph_objects as go
import pandas as pd

# Nastavenie parametrov

In [2]:
k = 7          # sucet kolkych akcii chceme zhora ohranicit
n = 45          # pocet akcii
alpha = 0.4     # horne ohranicenie pre lubovolnych k akcii

# Vynosy

# Nacitanie dat z Yahoo

In [3]:
data_file = 'sap100_data_08112021.csv'
sap100 = pd.read_csv(data_file, index_col = 0)
sap100.head()

,AAPL,ABBV,ABT,ACN,ADBE,AIG,AMGN,AMT,AMZN,AVGO,...,UNH,UNP,UPS,USB,V,VZ,WBA,WFC,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,128.997803,100.585396,107.449150,253.328720,485.339996,36.442303,223.416199,216.416885,3186.629883,418.644897,...,345.778137,200.139648,161.141953,45.091583,216.811539,56.299316,40.658897,29.300709,144.817017,39.621368
2021-01-05,130.592697,101.625504,108.778595,254.770905,485.690002,37.050823,224.500443,217.510788,3218.510010,421.480347,...,341.128174,202.594452,158.898941,45.238235,213.575684,56.041016,40.423191,30.119549,144.046143,41.530834
2021-01-06,126.196747,100.747612,108.552094,257.556427,466.309998,39.465260,229.911880,211.775146,3138.379883,418.881165,...,355.444122,206.833649,158.426727,47.917137,211.693924,56.720249,42.259716,32.250507,144.945511,42.590584
2021-01-07,130.502991,101.825890,109.605812,259.986450,477.739990,40.132664,230.670868,214.347321,3162.159912,436.533966,...,361.142761,209.712357,156.419815,48.875290,212.878738,56.592613,44.449795,32.980560,144.935623,42.924740
2021-01-08,131.629379,102.360252,109.911087,260.934753,485.100006,39.867664,235.076889,218.979187,3182.699951,438.749146,...,359.520264,215.726120,155.819717,48.220222,214.511597,55.886776,44.400692,32.743786,144.915848,43.402103


In [4]:
wanted_stocks = ['GOOG', 'SPG', 'GOOGL', 'MSFT', 'GD', 'ACN', 'COP', 'F', 'BAC', 'GS',
       'NVDA', 'AIG', 'MS', 'WFC', 'ORCL', 'XOM', 'TGT', 'LOW', 'EXC', 'COST',
       'AXP', 'BK', 'JPM', 'COF', 'CSCO', 'DHR', 'UNH', 'CVS', 'LLY', 'CVX',
       'MET', 'AMT', 'CRM', 'BLK', 'RTX', 'MCD', 'TMO', 'LIN', 'ADBE', 'EMR',
       'USB', 'UPS', 'TSLA', 'PFE', 'PM']
sap100 = sap100.loc[:,wanted_stocks]
sap100.head()

,GOOG,SPG,GOOGL,MSFT,GD,ACN,COP,F,BAC,GS,...,MCD,TMO,LIN,ADBE,EMR,USB,UPS,TSLA,PFE,PM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,1728.239990,80.108383,1726.130005,216.689423,142.191360,253.328720,38.381340,8.52,29.604614,261.802551,...,206.653915,471.064941,255.930817,485.339996,76.753830,45.091583,161.141953,729.770020,35.423965,78.486618
2021-01-05,1740.920044,80.456642,1740.050049,216.898438,142.259567,254.770905,40.583153,8.65,29.831354,267.660980,...,207.892548,476.955505,259.945679,485.690002,78.082092,45.238235,158.898941,735.109985,35.789654,79.343712
2021-01-06,1735.290039,82.294655,1722.880005,211.274414,145.893768,257.556427,42.202988,8.84,31.694584,282.104553,...,207.420670,484.433533,268.479645,466.309998,80.709106,47.917137,158.426727,755.979980,35.481701,79.449646
2021-01-07,1787.250000,82.352692,1774.339966,217.286652,147.822922,259.986450,43.357246,9.06,32.394527,288.130981,...,208.384064,499.529297,267.708313,477.739990,80.748459,48.875290,156.419815,816.039978,35.664547,79.911911
2021-01-08,1807.209961,83.668327,1797.829956,218.610550,146.897308,260.934753,43.357246,9.00,32.069199,286.579926,...,212.208069,511.480133,267.391876,485.100006,80.502480,48.220222,155.819717,880.020020,35.731911,80.017830


# Vyhodnotenie akcii

In [5]:
num_days = 252
rates = sap100/sap100.shift(1) -1
rates.head()

,GOOG,SPG,GOOGL,MSFT,GD,ACN,COP,F,BAC,GS,...,MCD,TMO,LIN,ADBE,EMR,USB,UPS,TSLA,PFE,PM
Date,,,,,,,,,,,,,,,,,,,,,
2021-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-01-05,0.007337,0.004347,0.008064,0.000965,0.000480,0.005693,0.057367,0.015258,0.007659,0.022377,...,0.005994,0.012505,0.015687,0.000721,0.017305,0.003252,-0.013919,0.007317,0.010323,0.010920
2021-01-06,-0.003234,0.022845,-0.009868,-0.025929,0.025546,0.010933,0.039914,0.021965,0.062459,0.053962,...,-0.002270,0.015679,0.032830,-0.039902,0.033644,0.059218,-0.002972,0.028390,-0.008605,0.001335
2021-01-07,0.029943,0.000705,0.029869,0.028457,0.013223,0.009435,0.027350,0.024887,0.022084,0.021362,...,0.004645,0.031162,-0.002873,0.024512,0.000488,0.019996,-0.012668,0.079447,0.005153,0.005818
2021-01-08,0.011168,0.015976,0.013239,0.006093,-0.006262,0.003648,0.000000,-0.006623,-0.010043,-0.005383,...,0.018351,0.023924,-0.001182,0.015406,-0.003046,-0.013403,-0.003836,0.078403,0.001889,0.001325


Ocakavene vynosy

In [6]:
V = rates.cov() * num_days
V.head()

,GOOG,SPG,GOOGL,MSFT,GD,ACN,COP,F,BAC,GS,...,MCD,TMO,LIN,ADBE,EMR,USB,UPS,TSLA,PFE,PM
GOOG,0.059152,0.015217,0.059287,0.035130,0.007999,0.019704,0.016042,0.011181,0.008179,0.009389,...,0.013291,0.013036,0.016222,0.033083,0.013596,0.003844,0.009888,0.038670,0.001664,0.008311
SPG,0.015217,0.092785,0.014787,0.006908,0.022520,0.016239,0.051497,0.055850,0.038585,0.040722,...,0.013736,-0.007785,0.020683,0.005540,0.032532,0.038065,0.013107,0.032465,0.002207,0.011899
GOOGL,0.059287,0.014787,0.062428,0.036748,0.007803,0.019641,0.016128,0.011420,0.007795,0.009054,...,0.012829,0.014384,0.016627,0.034359,0.014106,0.003055,0.009954,0.039171,0.003474,0.008258
MSFT,0.035130,0.006908,0.036748,0.042574,0.003716,0.019892,-0.002032,0.003265,-0.004523,-0.000584,...,0.010636,0.021209,0.015375,0.036231,0.007909,-0.005767,0.010340,0.042308,0.003970,0.007999
GD,0.007999,0.022520,0.007803,0.003716,0.029850,0.012099,0.021925,0.018899,0.023948,0.021352,...,0.008052,-0.003556,0.010943,-0.000334,0.022310,0.024644,0.008307,-0.005079,0.004833,0.008683


In [7]:
symbols = np.array(rates.columns)
r = rates.mean() * num_days
r = r.to_numpy()
len(r)

45

# Priprava matic a vektorov pre LP

In [8]:
c = np.zeros(2*n+1)
c[:n] = r

A_ub = np.zeros((n+1,2*n+1))
A_ub[0,n:-1] = np.ones(n)
A_ub[0,-1] = k
A_ub[1:,:n] = np.identity(n)
A_ub[1:,n:-1] = -np.identity(n)
A_ub[1:,-1] = -np.ones(n)

b_ub = np.zeros(n+1)
b_ub[0] = alpha

A_eq = np.zeros((1,2*n+1))
A_eq[0,:n] = np.ones(n)

b_eq = np.array([1])

x_bounds = [(0,None) for _ in range(n)]
lambda1_bounds = [(0,None) for _ in range(n)]
lambda2_bounds = [(None,None)]

bounds = []
for l in [x_bounds,lambda1_bounds,lambda2_bounds]:
    bounds.extend(l)

# Uloha LP

In [9]:
res = linprog(-c, A_ub=A_ub, b_ub=b_ub,A_eq =A_eq,b_eq=b_eq ,bounds=bounds,method='revised simplex')
print(res)
x = res.x[:n]

     con: array([3.33066907e-16])
     fun: -0.6818721944287918
 message: 'Optimization terminated successfully.'
     nit: 28
   slack: array([ 5.55111512e-17,  0.00000000e+00,  0.00000000e+00, -6.93889390e-18,
       -6.93889390e-18,  4.28571429e-02,  4.28571429e-02, -6.93889390e-18,
        6.93889390e-18, -6.93889390e-18, -6.93889390e-18, -6.93889390e-18,
       -6.93889390e-18, -6.93889390e-18, -6.93889390e-18, -6.93889390e-18,
       -6.93889390e-18, -6.93889390e-18, -6.93889390e-18,  4.28571429e-02,
        4.28571429e-02, -6.93889390e-18,  6.93889390e-18,  4.28571429e-02,
       -6.93889390e-18,  4.28571429e-02,  4.28571429e-02,  4.28571429e-02,
        4.28571429e-02,  6.93889390e-18,  4.28571429e-02,  4.28571429e-02,
        4.28571429e-02,  4.28571429e-02,  4.28571429e-02,  4.28571429e-02,
        4.28571429e-02,  4.28571429e-02,  4.28571429e-02,  4.28571429e-02,
        4.28571429e-02,  4.28571429e-02,  4.28571429e-02, -6.93889390e-18,
        4.28571429e-02,  4.28571429e-0

# Vizualizacia vysledkov

In [14]:
fig = go.Figure([go.Bar(x=wanted_stocks, y=x, marker_color = '#88bf4d')])
fig.update_layout(
    title="Váhy jednotlivých aktív (k = 7, alpha = 0.4)",
    xaxis_title="Symbol",
    yaxis_title="Percentuálny podiel",
)
fig.show()

In [15]:
colors = ['lightslategray',] * 45
for i in range(45):
    if x[i] > 0:
        colors[i] = '#88bf4d'
fig = go.Figure([go.Bar(x=wanted_stocks, y=r, marker_color=colors)])
fig.update_layout(
    title="Výnosy aktív (k = 7, alpha = 0.4)",
    xaxis_title="Symbol",
    yaxis_title="Očakávaný ročný výnos",
    
)
fig.show()

Vynos portfolia

In [12]:
final_r = np.sum(x * r)
final_r

0.6818721944287919

In [13]:
final_v = np.sqrt(np.dot(x.T, np.dot(V,x)))
final_v

0.17767685102018296

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=7d3ffbbf-2bc1-4a39-9f8f-ab5868813b8e' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>